In [1]:
from typing import List, Optional, Dict
import uuid
import pydicom
from fhir.resources.patient import Patient
from fhir.resources.observation import Observation
from fhir.resources.diagnosticreport import DiagnosticReport
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.reference import Reference
from datetime import datetime
import highdicom as hd

In [2]:
def recursive_search_for_coded_concept(
        ds: pydicom.Dataset, 
        coded_concept: hd.sr.CodedConcept
) -> List[pydicom.Dataset]:
    return_items = []
    for content_item in ds.ContentSequence:
        if hasattr(content_item, 'ConceptNameCodeSequence'):
            concept = content_item.ConceptNameCodeSequence[0]

            concept = hd.sr.CodedConcept(
                value=concept.CodeValue,
                scheme_designator=concept.CodingSchemeDesignator,
                meaning=concept.CodeMeaning
            )

            if concept == coded_concept:
                return_items.append(content_item)
        
        if hasattr(content_item, 'ContentSequence'):
            _return_items = recursive_search_for_coded_concept(content_item, coded_concept=coded_concept)
            return_items += _return_items
    return return_items


def create_observation_resource(
        code_value: str, 
        code_system: str, 
        code_display: str, 
        patient_id: str,
        value_quantity: Optional[Dict] = None
) -> Observation:
    return Observation(
        id=str(uuid.uuid4()),
        status="final",
        code=CodeableConcept(
            coding=[Coding(
                system=code_system,
                code=code_value,
                display=code_display
            )]
        ),
        subject=Reference(reference=f"Patient/{patient_id}"),
        effectiveDateTime=datetime.now().isoformat(),
        valueQuantity=value_quantity
    )


def string_observation(
        ds: pydicom.Dataset, 
        coded_concept: hd.sr.CodedConcept
) -> List[Observation]:
    items = recursive_search_for_coded_concept(ds, coded_concept)
    observations = []
    for item in items:
        item = item.ConceptCodeSequence[0]
        observation = create_observation_resource(
            code_value=item.CodeValue,
            code_system=item.CodingSchemeDesignator,
            code_display=item.CodeMeaning,
            patient_id=ds.PatientID
        )
        observations.append(observation)

    return observations


def numeric_measurements(
        ds: pydicom.Dataset, 
        coded_concept: hd.sr.CodedConcept
) -> List[Observation]:
    items = recursive_search_for_coded_concept(ds, coded_concept)
    observations = []
    for item in items:
        measurement = item.MeasuredValueSequence[0].NumericValue
        unit = item.MeasuredValueSequence[0].MeasurementUnitsCodeSequence[0]
        value_quantity = {
            "value": float(measurement),
            "unit": unit.CodeMeaning,
            "system": unit.CodingSchemeDesignator,
            "code": unit.CodeMeaning
        }
        observation = create_observation_resource(
            code_value=coded_concept.CodeValue,
            code_system=coded_concept.CodingSchemeDesignator,
            code_display=coded_concept.CodeMeaning,
            patient_id=ds.PatientID,
            value_quantity=value_quantity
        )
        observations.append(observation)

    return observations

In [3]:
ds = pydicom.dcmread('example_ecg_sr.dcm')
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Comprehensive SR Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.10.511.3.56431750657457273228940717842343408
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.9.7433.1.1
(0002, 0013) Implementation Version Name         SH: 'highdicom0.21.1'
-------------------------------------------------
(0008, 0016) SOP Class UID                       UI: Comprehensive SR Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.10.511.3.56431750657457273228940717842343408
(0008, 0020) Study Date                          DA: '20240101'
(0008, 0023) Content Date                        DA: '20240902'
(0008, 0030) Study Time 

In [4]:
# Convert DICOM Patient Information to FHIR Patient Resource
patient = Patient(
    id=ds.PatientID,
    name=[{
        "family": ds.PatientName.family_name,
        "given": ds.PatientName.given_name.split('^')
    }],
    birthDate=ds.PatientBirthDate,
    gender="male" if ds.PatientSex == "M" else "female"
)

In [5]:
dbp_code = hd.sr.CodedConcept(
    value='271650006',
    scheme_designator='SCT',
    meaning='Diastolic blood pressure (observable entity)'
)
dbp_observations = numeric_measurements(ds, dbp_code)

sbp_code = hd.sr.CodedConcept(
    value="271649006",
    scheme_designator='SCT',
    meaning="Systolic blood pressure (observable entity)"
)
sbp_oberservations = numeric_measurements(ds, sbp_code)


In [6]:
surgical_procedure_code = hd.sr.CodedConcept(
    value='387713003', 
    scheme_designator='SCT',
    meaning='Surgical Procedure'
)

surgical_procedure_observations = string_observation(ds, surgical_procedure_code)

In [7]:
observations = [
    *dbp_observations, *sbp_oberservations, *surgical_procedure_observations
]

In [8]:
# # Convert DICOM SR Document to FHIR DiagnosticReport
diagnostic_report = DiagnosticReport(
    status="final",
    code=CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code="11524-0",
            display="ECG Report"
        )]
    ),
    subject=Reference(reference=f"Patient/{ds.PatientID}"),
    effectiveDateTime=datetime.now().isoformat(),
    result=[Reference(reference=f"Observation/{obs.id}") for obs in observations]
)

In [9]:
diagnostic_report.dict()

OrderedDict([('resourceType', 'DiagnosticReport'),
             ('status', 'final'),
             ('code',
              OrderedDict([('coding',
                            [OrderedDict([('system', 'http://loinc.org'),
                                          ('code', '11524-0'),
                                          ('display', 'ECG Report')])])])),
             ('subject', OrderedDict([('reference', 'Patient/0123456789')])),
             ('effectiveDateTime',
              datetime.datetime(2024, 9, 3, 17, 58, 38, 238806)),
             ('result',
              [OrderedDict([('reference',
                             'Observation/2e5dd4ea-e4b2-4301-88f2-74a430c3c5e3')]),
               OrderedDict([('reference',
                             'Observation/d5dca584-85f7-4462-af47-7d0e3a968fc8')]),
               OrderedDict([('reference',
                             'Observation/9347876e-0ba2-4bdb-b547-0386bc1fe714')]),
               OrderedDict([('reference',
                   